In [1]:
import pandas as pd
import numpy as np 
import mlflow
import tensorflow
from tensorflow import keras
import mlflow.keras 
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.model_selection import train_test_split

df = pd.read_csv("../../data/training_data/training_data.csv")
df.head()

df = df.astype(np.float32)
X = df.iloc[:,:-1]
Y = df.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.33,
                                                    random_state=4284, stratify=Y)

In [3]:
mlflow.set_experiment("Baseline_Predictions_MLflow_Check")
mlflow.tensorflow.autolog()
input_ = keras.layers.Input(shape=X_train.shape[-1])
layer1 = keras.layers.Dense(units=10, activation='relu')(input_)
norm = keras.layers.BatchNormalization()(layer1)
output = keras.layers.Dense(units=1, activation='sigmoid')(norm)
model= keras.Model(input_, output)
model.compile(optimizer=keras.optimizers.Adam(lr=0.001),
             loss="binary_crossentropy",
             metrics="Accuracy")
with mlflow.start_run(run_name='keras_model_baseline') as run:
    model.fit(
        X_train,
        y_train,
        epochs=20,
        validation_split=0.05,
        shuffle=True
    )


INFO: 'Baseline_Predictions_MLflow_Check' does not exist. Creating a new experiment


/opt/conda/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/20
46/46 [==============================] - 1s 6ms/step - loss: 0.8197 - accuracy: 0.0000e+00 - val_loss: 0.7130 - val_accuracy: 0.0000e+00
Epoch 2/20
46/46 [==============================] - 0s 1ms/step - loss: 0.7662 - accuracy: 0.0000e+00 - val_loss: 0.6954 - val_accuracy: 0.0000e+00
Epoch 3/20
46/46 [==============================] - 0s 1ms/step - loss: 0.7317 - accuracy: 0.0000e+00 - val_loss: 0.6904 - val_accuracy: 0.0000e+00
Epoch 4/20
46/46 [==============================] - 0s 1ms/step - loss: 0.7200 - accuracy: 0.0000e+00 - val_loss: 0.6915 - val_accuracy: 0.0000e+00
Epoch 5/20
46/46 [==============================] - 0s 1ms/step - loss: 0.7073 - accuracy: 0.0000e+00 - val_loss: 0.6961 - val_accuracy: 0.0000e+00
Epoch 6/20
46/46 [==============================] - 0s 1ms/step - loss: 0.7002 - accuracy: 0.0000e+00 - val_loss: 0.7013 - val_accuracy: 0.0000e+00
Epoch 7/20
46/46 [==============================] - 0s 1ms/step - loss: 0.6953 - accuracy: 0.0000e+00 - val_loss

## Load the model for prediction

In [4]:
import mlflow 
logged_model = "/data/artifacts/4/3a2972e76a354feeb9a43c962c8336f7/artifacts/model"

# Load the model as a pyfunc model
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a pandas dataframe
loaded_model.predict(X_test)

/opt/conda/lib/python3.8/site-packages/tensorflow/python/keras/backend.py:435: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


,0
874,0.382366
1151,0.638954
912,0.563962
2071,0.589712
985,0.589581
...,...
2222,0.615050
524,0.629220
2012,0.535705
119,0.809855
